In [10]:
import os
from ultralytics import YOLO
from paddleocr import PaddleOCR
from PIL import Image
import numpy as np

In [8]:
image = 'D:\\ProgPrj\\\mezhnar\\data\\data_to_mix\\imgs\\21.JPG'
model_path = 'D:\\ProgPrj\\mezhnar\\research\\pipeline\\best.pt'
output_folder = 'D:\\ProgPrj\\mezhnar\\research\\pipeline'

In [4]:
model_pt = YOLO(model_path)

In [6]:
results = model_pt(image)


image 1/1 D:\ProgPrj\mezhnar\data\data_to_mix\imgs\21.JPG: 640x480 1 0, 139.8ms
Speed: 13.0ms preprocess, 139.8ms inference, 608.0ms postprocess per image at shape (1, 3, 640, 480)


In [9]:
import os
import math
import numpy as np
from PIL import Image

os.makedirs(output_folder, exist_ok=True)

best_confidence = -1  # Начальное значение для хранения лучшей вероятности
best_cropped_image = None  # Переменная для хранения лучшего обрезанного изображения

for result in results:
    original_image = result.orig_img  # Получаем оригинальное изображение без bbox

    # Проверяем, есть ли у result атрибут boxes и обрабатываем bbox, если он есть
    if hasattr(result, 'boxes'):
        total_boxes = len(result.boxes)
        print(f"Total bounding boxes: {total_boxes}")

        for i, box in enumerate(result.boxes):
            # Проверяем, что box имеет атрибут xyxy для извлечения координат
            if hasattr(box, 'xyxy') and box.xyxy.shape[-1] == 4:
                x_min, y_min, x_max, y_max = box.xyxy[0].tolist()

                # Попытка получить значение вероятности
                confidence = None
                if hasattr(box, 'confidence'):
                    confidence = box.confidence.item() * 100
                elif hasattr(box, 'score'):
                    confidence = box.score.item() * 100
                elif hasattr(box, 'conf'):
                    confidence = box.conf.item() * 100

                # Проверяем, что значение confidence не NaN и больше текущего лучшего значения
                if confidence is not None and not math.isnan(confidence):
                    print(f"Bounding Box {i + 1} Probability: {confidence:.2f}%")

                    if confidence > best_confidence:
                        best_confidence = confidence

                        # Расширяем рамки с учетом padding и обрезаем изображение
                        padding = 0.1  # Процент увеличения рамки
                        height, width, _ = original_image.shape
                        x_min = max(0, int(x_min - padding * (x_max - x_min)))
                        y_min = max(0, int(y_min - padding * (y_max - y_min)))
                        x_max = min(width, int(x_max + padding * (x_max - x_min)))
                        y_max = min(height, int(y_max + padding * (y_max - y_min)))

                        # Обрезаем изображение по расширенным рамкам и сохраняем как лучшее
                        best_cropped_image = original_image[y_min:y_max, x_min:x_max]
                else:
                    print(f"Ошибка: Probability для Bounding Box {i + 1} является NaN или отсутствует.")
    else:
        print("Ошибка: result не содержит атрибут boxes.")

# Сохраняем оригинал и три поворота, если изображение было найдено
if best_cropped_image is not None:
    best_cropped_image_pil = Image.fromarray(best_cropped_image)
    best_cropped_image_pil.save(os.path.join(output_folder, "original.png"))

    for rotation in range(1, 4):
        rotated_image = np.rot90(best_cropped_image, rotation)
        rotated_image_pil = Image.fromarray(rotated_image)
        rotated_image_pil.save(os.path.join(output_folder, f"rotated_{rotation * 90}.png"))
else:
    print("Нет подходящего изображения с уверенностью выше порога.")


Total bounding boxes: 1
Bounding Box 1 Probability: 85.48%


In [11]:
ocr_paddle = PaddleOCR(use_angle_cls=True, lang='en')

[2024/11/09 22:09:17] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Geo/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Geo/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, m

In [14]:
# Проход по всем файлам в папке
for filename in os.listdir(output_folder):
    if filename.endswith('.png') or filename.endswith('.jpg'):  # Добавьте другие форматы, если необходимо
        file_path = os.path.join(output_folder, filename)

        # Загрузка изображения и преобразование в numpy массив
        image = Image.open(file_path)
        image_np = np.array(image)

        # Выполнение OCR
        results = ocr_paddle.ocr(image_np, cls=True)

        # Проверка, есть ли результаты
        if results and results[0]:
            print(f"Результаты для {filename}:")
            for line in results[0]:
                text = line[1][0]
                confidence = line[1][1]
                print(f"text: {text}, confidence: {confidence}")
        else:
            print(f"На изображении {filename} текст не найден.")

        print("\n" + "="*50 + "\n")  # Разделитель между изображениями

[2024/11/09 22:13:41] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.02778458595275879
[2024/11/09 22:13:41] ppocr DEBUG: cls num  : 2, elapsed : 0.005000591278076172
[2024/11/09 22:13:41] ppocr DEBUG: rec_res num  : 2, elapsed : 0.04700303077697754
Результаты для original.png:
text: 195-30-128, confidence: 0.7270439267158508
text: 2652, confidence: 0.7332246899604797


[2024/11/09 22:13:41] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.026001453399658203
[2024/11/09 22:13:41] ppocr DEBUG: cls num  : 2, elapsed : 0.007002115249633789
[2024/11/09 22:13:41] ppocr DEBUG: rec_res num  : 2, elapsed : 0.04859042167663574
На изображении rotated_180.png текст не найден.


[2024/11/09 22:13:41] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.1371452808380127
[2024/11/09 22:13:41] ppocr DEBUG: cls num  : 2, elapsed : 0.005001068115234375
[2024/11/09 22:13:41] ppocr DEBUG: rec_res num  : 2, elapsed : 0.07559084892272949
Результаты для rotated_270.png:
text: 195-30-1286, confidence: 0.7501741051673889
text: 2

In [29]:
import pandas as pd
from Levenshtein import distance as levenshtein_distance

data = 'D:\\ProgPrj\\mezhnar\\data\\ДеталиПоПлануДляРазрешенныхЗаказов.xlsx'
df = pd.read_excel(data)

# OCR результаты
ocr_detaliartikuly = ["195-30-128", "195-30-1286", "95-30-1280"]
ocr_poryadkovy_nomer = "2652"

# Порог вероятности для соответствия по расстоянию Левенштейна
threshold = 6 # Можно настроить в зависимости от требуемой точности

# Функция для поиска строк с заданным порогом по расстоянию Левенштейна
def find_similar_rows(df, ocr_artikuls, ocr_poryadkovy, threshold):
    results = []

    for ocr_artikul in ocr_artikuls:
        for index, row in df.iterrows():
            # Преобразуем значения в строку
            artikul_value = str(row['ДетальАртикул'])
            poryadkovy_value = str(row['ПорядковыйНомер'])

            artikul_distance = levenshtein_distance(ocr_artikul, artikul_value)
            poryadkovy_distance = levenshtein_distance(ocr_poryadkovy, poryadkovy_value)

            if artikul_distance <= threshold and poryadkovy_distance <= threshold:
                results.append({
                    "index": index,
                    "ДетальАртикул": artikul_value,
                    "ПорядковыйНомер": poryadkovy_value,
                    "АртикулСовпадение": 100 - (artikul_distance / max(len(ocr_artikul), len(artikul_value))) * 100,
                    "ПорядковыйСовпадение": 100 - (poryadkovy_distance / max(len(ocr_poryadkovy), len(poryadkovy_value))) * 100
                })

    return pd.DataFrame(results)

# Выполнение поиска и вывод результатов
similar_rows = find_similar_rows(df, ocr_detaliartikuly, ocr_poryadkovy_nomer, threshold)

In [30]:
similar_rows

,index,ДетальАртикул,ПорядковыйНомер,АртикулСовпадение,ПорядковыйСовпадение
0,3073,"""1753-30-0120""",0.0,57.142857,0.0
1,3083,"""1753-30-0129""",0.0,57.142857,0.0
2,3086,"""1753-30-0142""",0.0,57.142857,0.0
3,3096,"""1753-30-1062""",0.0,57.142857,0.0
4,3097,"""1753-30-1324""",0.0,57.142857,0.0
...,...,...,...,...,...
211,4428,"""1753-30-0129""",0.0,57.142857,0.0
212,4442,"""1753-30-1324""",0.0,57.142857,0.0
213,4443,"""1753-30-1325""",0.0,57.142857,0.0
214,4444,"""1753-30-1326""",0.0,57.142857,0.0
